In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.pyplot import figure
from IPython.display import clear_output

In [ ]:
# Import dataset
raw_data = pd.read_csv("../../datasets/raw/fire_nrt_V1_95405.csv") 
raw_data.head()

In [ ]:
def select_timeframe(dataframe, start, end):
    """
    select a start and an end date (int) of the 92 days of available data
    """
    
    assert end - start > 0    
    date_range = dataframe.acq_date.unique()[start : end]
    
    return dataframe.loc[dataframe['acq_date'].isin(date_range)].reset_index()

In [ ]:
def normalize_coordinates(input_data, x_scale, y_scale, max_lat, min_lat, max_lon, min_lon):
    lat = input_data.latitude
    lon = input_data.longitude
    
    # Normalize coordinates
    df = pd.DataFrame()
    df['lat'] = (lat - min_lat) / (max_lat - min_lat)
    df['lon'] = (lon - min_lon) / (max_lon - min_lon)

    df['lon'] = df.lon * (x_scale - 1)
    df['lat'] = df.lat * (y_scale - 1)

    # Round
    df['lon'] = round(df.lon)
    df['lat'] = round(df.lat)
    return df

In [ ]:
def construct_heatmap(dataframe, in_x, in_y, out_x, out_y, scale_on=True):
    im = np.zeros((in_y, in_x))
    heat_range = len(dataframe)
    
    for i in range(heat_range):
        if im[int(dataframe.lat[i]), int(dataframe.lon[i])] == 0:
            im[int(dataframe.lat[i]), int(dataframe.lon[i])] = 1

        else:
            if scale_on is True:
                im[int(dataframe.lat[i]), int(dataframe.lon[i])] += 0

    # Add ocean buffer
    bottom_ocean = np.zeros((int((in_x - in_y) / 2), in_x))
    height = (int((in_x - in_y) / 2) + in_y)
    side_ocean = np.zeros((height, int(height * 50/1000)))
    
    # bottom top
    im1 = np.concatenate((bottom_ocean, im), 0)

    # left right
    im1 = np.concatenate((side_ocean, im1, side_ocean), 1)
    
    im1 = cv2.resize(im1, (out_x, out_y), interpolation=cv2.INTER_NEAREST)
    return im1

In [ ]:
def plot_data(data, mask_image, date):
    shape = np.shape(data)    
    mask_image = cv2.resize(mask_image, (shape[1], shape[0]), interpolation=cv2.INTER_NEAREST)
    
    assert np.shape(data) == np.shape(mask_image)

    figure(num=None, figsize=(16, 16))
    im = np.ma.masked_where(mask_image == 0, data)
    
    # Plot
    cmap = plt.cm.OrRd
    plt.imshow(im, cmap='plasma', interpolation='nearest', origin='lower', vmin=0, vmax=3)
    plt.colorbar()
    plt.title(date)
    plt.show()
    
    clear_output(wait=True)

In [ ]:
def animate(raw_data, mask, x_in, y_in, x_out, y_out, iterations):
    to_plot = None
    dates = raw_data.acq_date.unique()
    
    # Set extrema
    lat = raw_data.latitude
    lon = raw_data.longitude

    max_lat = max(lat)
    min_lat = min(lat)
    max_lon = max(lon)
    min_lon = min(lon)
    
    
    for i in range(iterations):
        
        step1 = select_timeframe(raw_data, i, i+1)
        step2 = normalize_coordinates(step1, x_in, y_in, max_lat, min_lat, max_lon, min_lon)
        step3 = construct_heatmap(step2, x_in, y_in, x_out, y_out, False)
        
        if to_plot is None:
            to_plot = step3
        else:
            to_plot *= 0.90
            to_plot += step3
        
        plot_data(to_plot, mask, dates[i])
        
        print(max_lat, min_lat, max_lon, min_lon)

        
        
        
mask = np.load('australia-mask.npy')

x_in = 220
y_in = 180
x_out = 1100
y_out = 900
        
animate(raw_data, mask, x_in, y_in, x_out, y_out, 93)

In [ ]:
# # add vegetation to australia

# figure(num=None, figsize=(16, 16))

# im = np.load('australia-vegetation-heatmap.npy')

# im = np.where(im != 0, im, 1)

# mask = cv2.resize(mask, (2200, 1800), interpolation=cv2.INTER_NEAREST)
# im = cv2.resize(im, (2200, 1800), interpolation=cv2.INTER_NEAREST)


# im = np.ma.masked_where(mask == 0, im)


# plt.imshow(im, cmap='plasma', interpolation='nearest', origin='lower')

# print(np.unique(im))

In [ ]:
import random

def add_vegetation(out_x, out_y):
    """
    Create a vegetated map in the form of a numpy array of specified dimension.
    A recommended ratio is 11/9    
    """
    # import vegetation image
    veg = np.load('../../datasets/processed/australia-vegetation.npy')

    # import mask image
    mask = np.load('../../datasets/processed/australia-mask.npy')

    out_im = np.zeros((out_y, out_x))
    
    # Mask 
    veg = np.where(veg != 0, veg, 1)    
    mask = cv2.resize(mask, (out_x, out_y), interpolation=cv2.INTER_NEAREST)
    veg = cv2.resize(veg, (out_x, out_y), interpolation=cv2.INTER_NEAREST)
    veg = np.ma.masked_where(mask == 0, veg)
    
    assert len(np.unique(veg)) == 6, 'Provide valid vegetation image'
    assert len(np.unique(mask)) == 2, 'Provide valid mask image'

    for (i, p) in [(1,0.1), (2,0.3), (3,0.5), (4,0.7), (5,0.9)]:
        for (x,y) in np.argwhere(veg == i):
            out_im[x, y] = (1 if random.random() < p else 2)
        
    return out_im

figure(num=None, figsize=(16, 16))
out = add_vegetation(220, 180)
plt.imshow(out, origin='lower')
plt.show()

np.save('../../datasets/processed/vegetation_array', out)